## Understanding Tensorflow : 

Tensorflow is a perfect open source library for research and production popularly used for deep learning. <br>
This notebook's goal is to see how tensorflow works under the hood. I will be trying to mimic its API and implement the core building blocks from scratch. The most important term I will be dealing with are variables, constants, tensors, sessions and operations. <br><br>
Tensorflow is composed of two core building blocks : <br> 
$\bullet$ A library for defining __computational graphs__ : an abstract way of describing computations as a directed graph. <br>
$\bullet$ A __runtime__ for execution.<br><br>

Computational graphs, also known as __data flow graphs__, have nodes that represent mostly operations, variables or placeholders and edges that represent data in the form of tensors. <br> <br>
Using computational graphs allow parallelism or dependency driving scheduling between operations if the latter do not depend on each other even though they have same input. ==> More efficiency 

Tensorflow separates declaration and execution (Better than numpy since it doesn't directly allocate memory, less errors).<br><br>
For now, I am going to implement different classes to try to mimic tensorflow's API : Graph, Operation, add, multiply, divide, matmul, Placeholder, Constant, Variable, Session 

In [1]:
import numpy as np

In [2]:
class Graph():
    def __init__(self):
        self.operations = []
        self.placeholders = []
        self.variables = []
        self.constants = []

    def as_default(self):
        global _default_graph
        _default_graph = self

In [3]:
class Operation():
    def __init__(self, input_nodes=None):
        self.input_nodes = input_nodes
        self.output = None
    
    # An operation adds itself to the default graph
        _default_graph.operations.append(self)

    def forward(self):
        pass

In [4]:
class BinaryOperation(Operation):
    def __init__(self, a, b):
        super().__init__([a, b])

In [5]:
class add(BinaryOperation):
    """
    Computes a + b, element-wise
    """
    def forward(self, a, b):
        return a + b


class multiply(BinaryOperation):
    """
    Computes a * b, element-wise
    """
    def forward(self, a, b):
        return a * b


class divide(BinaryOperation):
    """
    Returns the true division of the inputs, element-wise
    """
    def forward(self, a, b):
        return np.true_divide(a, b)


class matmul(BinaryOperation):
    """
    Multiplies matrix a by matrix b, producing a * b
    """
    def forward(self, a, b):
        return a.dot(b)

In [6]:
class Placeholder():
    def __init__(self):
        self.value = None
        _default_graph.placeholders.append(self)

In [7]:
class Constant():
    def __init__(self, value=None):
        self.__value = value
        _default_graph.constants.append(self)

    @property
    def value(self):
        return self.__value

    @value.setter
    def value(self, value):
        raise ValueError("Cannot reassign value.")

In [8]:
class Variable():
    def __init__(self, initial_value=None):
        self.value = initial_value
        _default_graph.variables.append(self)

In [9]:
def topology_sort(operation):
    ordering = []
    visited_nodes = set()

    def recursive_helper(node):
        if isinstance(node, Operation):
            for input_node in node.input_nodes:
                if input_node not in visited_nodes:
                    recursive_helper(input_node)

        visited_nodes.add(node)
        ordering.append(node)

    # start recursive depth-first search
    recursive_helper(operation)

    return ordering

In [10]:
class Session():
    def run(self, operation, feed_dict={}):
        nodes_sorted = topology_sort(operation)

        for node in nodes_sorted:
            if type(node) == Placeholder:
                node.output = feed_dict[node]
            elif type(node) == Variable or type(node) == Constant:
                node.output = node.value
            else:
                inputs = [node.output for node in node.input_nodes]
                node.output = node.forward(*inputs)

        return operation.output

Now that we have defined all the components of our API mimicing Tensorflow, let's test in on simple operation :

In [11]:
# Creating the default computational graph
Graph().as_default()

# Adding some nodes
a = Constant(7)
b = Constant(8)
prod = multiply(a, b)
sum = add(a, b)
res = divide(prod, sum)

# create a session object
session = Session()

# run computational graph to compute the output for 'res'
out = session.run(res)
print(out)

3.7333333333333334
